## Установка необходимых библиотек и загрузка данных

In [ ]:
!pip install map-boxes

In [ ]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import random

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r /content/yolov5/requirements.txt

In [4]:
!mkdir /content/participants

Загрузим датасеты в папку participants. Либо в ручном режиме, либо через curl, либо с Google Drive.



In [ ]:
!curl https://lodmedia.hb.bizmrg.com/case_files/789899/train_dataset_train.zip -o /content/participants/train.zip
!curl https://lodmedia.hb.bizmrg.com/case_files/789899/test_dataset_test.zip -o /content/participants/test.zip

Разархивируем их

In [10]:
!unzip -q /content/participants/train.zip -d /content/participants
!unzip -q /content/participants/test.zip -d /content/participants

Помотрим на представленные классы

In [ ]:
pd.Series([file_name.split('_')[-1][:-5] for file_name in os.listdir('participants/train/labels')]).value_counts()

## Подготовка данных

Во всех .txt файлах стоит класс 0. Поэтому выставим нужные классы, указанные в условиях задачи (car': 0, 'head': 1, 'face': 2, 'human': 3, 'carplate': 4)

In [12]:
for file_name in os.listdir("participants/train/labels"):
  file_class = file_name.split("_")[-1][:-5]
  file_path = "participants/train/labels/" + file_name
  f1 = open(file_path, "r")
  content = f1.readlines()
  f1.close()
  if file_class == "head":
    new_content = ["1" + line[1:] for line in content]
    f2 = open(file_path, "w")
    f2.writelines(new_content)
    f2.close()
  elif file_class == "human":
    new_content = ["3" + line[1:] for line in content]
    f2 = open(file_path, "w")
    f2.writelines(new_content)
    f2.close()
  elif file_class == "car":
    new_content = ["0" + line[1:] for line in content]
    f2 = open(file_path, "w")
    f2.writelines(new_content)
    f2.close()
  elif file_class == "face":
    new_content = ["2" + line[1:] for line in content]
    f2 = open(file_path, "w")
    f2.writelines(new_content)
    f2.close()
  elif file_class == "carplate":
    new_content = ["4" + line[1:] for line in content]
    f2 = open(file_path, "w")
    f2.writelines(new_content)
    f2.close()
  else:
    print("New class")

Создадим обучающий датасет в формате YOLO

In [13]:
!mkdir data_for_yolo
!mkdir data_for_yolo/data
!mkdir data_for_yolo/data/images
!mkdir data_for_yolo/data/labels
!mkdir data_for_yolo/data/images/train
!mkdir data_for_yolo/data/labels/train
!mkdir data_for_yolo/data/images/test
!mkdir data_for_yolo/data/labels/test

В папке data_for_yolo создадим dataset.yaml со следующим содержимым:

In [14]:
yaml_content = """
train: /content/data_for_yolo/data/images/train/
val: /content/data_for_yolo/data/images/test/

# number of classes
nc: 5

# class names
names: ['car', 'head', 'face', 'human', 'carplate']
"""

Создадим pandas DataFrame для удобного представления данных

In [ ]:
labels = pd.DataFrame(os.listdir("/content/participants/train/labels"), columns=["file name"])
labels["image name"] = labels["file name"].apply(lambda x: "_".join(x.split("_")[:-1]))
labels["label"] = labels["file name"].apply(lambda x: x.split("_")[-1][:-5])
labels.head()

Обнаружим и удалим дубликаты

In [ ]:
print("Found duplicates: ", labels["image name"].duplicated().sum())
images_names = labels["image name"].sort_values().drop_duplicates().tolist()

В исходном датасете для каждого обнаруженного объекта есть свой .txt файл разметки с одной строкой. Для модели YOLO необходимо, чтобы каждой картинке соответствовал один файл разметки. Этот файл должен содержать строки с обнаруженными объектами. Приведём файлы разметки к нужному формату:

In [17]:
!mkdir /content/participants/train/new_labels

In [ ]:
from tqdm import tqdm

for image_name in tqdm(images_names):
  f1 = open("./participants/train/new_labels/" + image_name + ".txt", "a+")
  for label_name in os.listdir("./participants/train/labels"):
    if image_name == "_".join(label_name.split("_")[:-1]):
      f2 = open("./participants/train/labels/" + label_name).readlines()
      f1.writelines(f2)
  f1.close()

In [19]:
new_labels_names = os.listdir("/content/participants/train/new_labels")

Разделим выборку на обучающую и проверочную. Это код для обучения модели, поэтому почти весь датасет используем в тренировочных целях (для теста остается 5% картинок)





In [20]:
train_images_names, test_images_names = train_test_split(new_labels_names, test_size=0.05, random_state=42)

In [ ]:
train_images_names[:5]

Переместим новые файлы разметки в папки для обучения

In [ ]:
#train
for train_image_label in tqdm(train_images_names):
  #labels
  shutil.copy('participants/train/new_labels/' + train_image_label, '/content/data_for_yolo/data/labels/train/' + train_image_label)
  #images
  train_image_name = train_image_label[:-4]
  shutil.copy('participants/train/images/' + train_image_name + '.jpg', '/content/data_for_yolo/data/images/train/')

#test
for test_image_label in tqdm(test_images_names):
  #labels
  shutil.copy('participants/train/new_labels/' + test_image_label, '/content/data_for_yolo/data/labels/test/' + test_image_label)
  #images
  test_image_name = test_image_label[:-4]
  shutil.copy('participants/train/images/' + test_image_name + '.jpg', '/content/data_for_yolo/data/images/test/')

## Обучение

Библиотека Albumentations для аугментации данных. Билиотека интегрирована с YOLO.

In [ ]:
!pip install -U albumentations

In [24]:
path_to_data = './data_for_yolo/dataset.yaml'

In [ ]:
!nvidia-smi

Для применения аугментации необходимо указать путь к гиперпараметрам (hyp.scratch-high.yaml). В этом файле содержаться параметры для обучения модели и аугментации. Дальше YOLO сделает всё сам.

In [ ]:
!python ./yolov5/train.py --img 640 --batch 16 --epochs 70 --data {path_to_data} --weights yolov5x6.pt --hyp ./yolov5/data/hyps/hyp.scratch-high.yaml --name yolov5x6_high_results --cache

## Результаты валидации на проверочной выборке

In [ ]:
!python3 ./yolov5/detect.py --weights ./yolov5/runs/train/yolov5x6_high_results/weights/best.pt --img 640 --conf 0.5 --source ./data_for_yolo/data/images/test --save-txt --save-conf

In [29]:
def get_soliton_labels_df(path_to_txt_folder):
  simple_solution = []
  for detection_file in os.listdir(path_to_txt_folder):
    img_name = detection_file.split('.')[0] + '.jpg'
    with open(path_to_txt_folder + detection_file, 'r') as f:
      data = f.read()
      data = [i for i in data.split('\n') if i != '']
    for line in data:
      val = [float(i) for i in line.split()]
      cls, xywh, conf = val[0], val[1:5], val[5]
      center_x, center_y, width, height = xywh
      xmin = center_x - (width / 2)
      xmax = center_x + (width / 2)
      ymin = center_y - (height / 2)
      ymax = center_y + (height / 2)
      simple_solution.append([img_name, cls, conf, xmin, xmax, ymin, ymax])
  return simple_solution

In [30]:
simple_solution = get_soliton_labels_df('yolov5/runs/detect/exp/labels/')
simple_solution = pd.DataFrame(simple_solution, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])

In [31]:
def get_test_labels_df(path_to_txt_folder):
  simple_solution = []
  for detection_file in os.listdir(path_to_txt_folder):
    img_name = detection_file.split('.')[0] + '.jpg'
    with open(path_to_txt_folder + detection_file, 'r') as f:
      data = f.read()
      data = [i for i in data.split('\n') if i != '']
    for line in data:
      val = [float(i) for i in line.split()]
      cls, center_x, center_y, width, height = val
      xmin = center_x - (width / 2)
      xmax = center_x + (width / 2)
      ymin = center_y - (height / 2)
      ymax = center_y + (height / 2)
      simple_solution.append([img_name, cls, xmin, xmax, ymin, ymax])
  return simple_solution

In [32]:
test_labels = get_test_labels_df('data_for_yolo/data/labels/test/')
test_labels = pd.DataFrame(test_labels, columns=['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax'])

In [33]:
mean_ap, average_precisions = mean_average_precision_for_boxes(test_labels, simple_solution, iou_threshold=0.5, verbose=False)

In [ ]:
print(mean_ap)